In [ ]:
import numpy as np
import tensorflow as tf
import os
from datetime import datetime

In [ ]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
# train_data = mnist.train.images # Returns np.array
# train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
X_test=mnist.test.images
y_test=mnist.test.labels

In [ ]:
n_inputs=28
# n_neurons=100
n_outputs=10
# n_rnn_layers=1
keep_prob=1
n_steps=28
n_epochs=100
batch_size=150
learning_rate=0.001


num_layers=2
num_neurons=[100,100,400,200]
optimizer_fn=tf.train.AdamOptimizer
now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs_rnn"
logdir="{}/run-{}/".format(root_logdir,now)
savedir='/tmp/my_rnn_model_final.ckpt'


# activation_func

In [ ]:
def build_RNN_graph():
    tf.reset_default_graph()
    he_init=tf.contrib.layers.variance_scaling_initializer()
    X=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
    Y=tf.placeholder(tf.int32,[None])
    global_step=tf.Variable(0,trainable=False,name='global_step')
    training=tf.placeholder_with_default(False,shape=(),name='training')
    
    cells=[tf.contrib.rnn.BasicLSTMCell(num_units=num_neurons[i])   for i in range(num_layers)]
#     if tf.cond(training):
#         cells=[tf.contrib.rnn.DropoutWrapper(cell,input_keep_prob=keep_prob) for cell in cells]
        
    multi_layer_cell=tf.contrib.rnn.MultiRNNCell(cells)
    rnn_outputs,states=tf.nn.dynamic_rnn(multi_layer_cell,X,dtype=tf.float32)

    logits=tf.layers.dense(states[-1][-1],n_outputs,kernel_initializer=he_init)
    entropies=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y,logits=logits)
    loss=tf.reduce_mean(entropies)
    
    optimizer=optimizer_fn(learning_rate=learning_rate)
    training_op=optimizer.minimize(loss,global_step=global_step)
    correct=tf.nn.in_top_k(logits,Y,1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

    init=tf.global_variables_initializer()
    
    loss_summary=tf.summary.scalar('loss',loss)
    acc_summary=tf.summary.scalar('accuracy',accuracy)
    summaries=tf.summary.merge_all()
    file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())
    saver=tf.train.Saver()
    
    return X,Y,init,loss,training_op,accuracy,global_step,summaries,file_writer,saver
    
    
    
    

In [ ]:
X_test=X_test.reshape((-1,n_steps,n_inputs))
X,Y,init,loss,training_op,accuracy,global_step,summaries,file_writer,saver=build_RNN_graph()
with tf.Session()as sess:  
    if os.path.isfile(savedir):
        saver.restore(sess,savedir)
    else:
        init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            step=global_step.eval()
            X_batch,y_batch=mnist.train.next_batch(batch_size)
            X_batch=X_batch.reshape((-1,n_steps,n_inputs))
            
            if iteration%10==0:
                summary_str=summaries.eval(feed_dict={X:X_test,Y:y_test})
                file_writer.add_summary(summary_str,step)
                
            sess.run(training_op,feed_dict={X:X_batch,Y:y_batch})
        acc_train=accuracy.eval(feed_dict={X:X_batch,Y:y_batch})
        acc_test=accuracy.eval(feed_dict={X:X_test,Y:y_test})

        print(epoch,"Train accuracy:",acc_train,"Test accuracy:",acc_test)
    if (epoch%5)==0:# checkpoint every 5 epochs
            save_path=saver.save(sess,savedir)
    file_writer.close()

In [ ]:
# ! tensorboard --logdir tf_logs_rnn